baxterのmap求める

In [65]:
import sympy as sy
from sympy import sin, cos, pi, sqrt


In [66]:
q = sy.Matrix(sy.MatrixSymbol('q', 7, 1))
L, h, H, L0, L1, L2, L3, L4, L5, L6, R = sy.symbols('L, h, H, L0, L1, L2, L3, L4, L5, L6, R')

def HTM(alpha, a, d, theta):
    return sy.Matrix([
        [cos(theta), -sin(theta), 0, a],
        [sin(theta)*cos(alpha), cos(theta)*cos(alpha), -sin(alpha), -d*sin(alpha)],
        [sin(theta)*sin(alpha), cos(theta)*sin(alpha), cos(alpha), d*cos(alpha)],
        [0, 0, 0, 1],
    ])



DHparams = (
    (0, 0, 0, q[0, 0]),
    (-pi/2, L1, 0, q[1, 0]+pi/2),
    (pi/2, 0, L2, q[2, 0]),
    (-pi/2, L3, 0, q[3, 0]),
    (pi/2, 0, L4, q[4, 0]),
    (-pi/2, L5, 0, q[5, 0]),
    (pi/2, 0, 0, q[6, 0]),
)


T_RL_W0 = sy.Matrix([
    [-sqrt(2)/2, sqrt(2)/2, 0, -L,],
    [-sqrt(2)/2, -sqrt(2)/2, 0, -h,],
    [0, 0, 1, H,],
    [0, 0, 0, 1,],
])

T_0_RL = sy.Matrix([
    [1, 0, 0, 0,],
    [0, 1, 0, 0,],
    [0, 0, 1, L0,],
    [0, 0, 0, 1,],
])

Ts = [HTM(*dhparam) for dhparam in DHparams]

T_GR_7 = sy.Matrix([
    [1, 0, 0, 0,],
    [0, 1, 0, 0,],
    [0, 0, 1, L6,],
    [0, 0, 0, 1,],
])


### 変換前一覧 ###
T_all = [T_RL_W0, T_0_RL]
T_all += Ts
T_all.append(T_GR_7)


### 変換後 ###
for i, T in enumerate(T_all):
    if i == 0:
        T_abs = [T]
    else:
        T_abs.append(T_abs[i-1] @ T)


os = [T[0:3, 3:4] for T in T_abs]
Rxs = [T[0:3, 0:1] for T in T_abs]
Rys = [T[0:3, 1:2] for T in T_abs]
Rzs = [T[0:3, 2:3] for T in T_abs]

Jos = [o.jacobian(q) for o in os]
JRxs = [r.jacobian(q) for r in Rxs]
JRys = [r.jacobian(q) for r in Rys]
JRzs = [r.jacobian(q) for r in Rzs]



t = sy.Symbol("t")
q1 = sy.Function("q1")
q2 = sy.Function("q2")
q3 = sy.Function("q3")
q4 = sy.Function("q4")
q5 = sy.Function("q5")
q6 = sy.Function("q6")
q7 = sy.Function("q7")

dq = sy.Matrix(sy.MatrixSymbol('dq', 7, 1))

T_abs_ = []
for T in T_abs:
    T_ = T.subs([
        (q[0,0], q1(t)),
        (q[1,0], q2(t)),
        (q[2,0], q3(t)),
        (q[3,0], q4(t)),
        (q[4,0], q5(t)),
        (q[5,0], q6(t)),
        (q[6,0], q7(t)),
    ])
    T_abs_.append(T_)


os_ = [T[0:3, 3:4] for T in T_abs_]
Rxs_ = [T[0:3, 0:1] for T in T_abs_]
Rys_ = [T[0:3, 1:2] for T in T_abs_]
Rzs_ = [T[0:3, 2:3] for T in T_abs_]

q_ = sy.Matrix([
    [q1(t)],
    [q2(t)],
    [q3(t)],
    [q4(t)],
    [q5(t)],
    [q6(t)],
    [q7(t)],
])
Jos_ = [o.jacobian(q_) for o in os_]
JRxs_ = [r.jacobian(q_) for r in Rxs_]
JRys_ = [r.jacobian(q_) for r in Rys_]
JRzs_ = [r.jacobian(q_) for r in Rzs_]

Jos_dot_ = [sy.diff(J, t) for J in Jos_]
JRxs_dot_ = [sy.diff(J, t) for J in JRxs_]
JRys_dot_ = [sy.diff(J, t) for J in JRys_]
JRzs_dot_ = [sy.diff(J, t) for J in JRzs_]


Jos_dot = []
JRxs_dot = []
JRys_dot = []
JRzs_dot = []
for Js, newJs in zip((Jos_dot_, JRxs_dot_, JRys_dot_, JRzs_dot_), (Jos_dot, JRxs_dot, JRys_dot, JRzs_dot)):
    for J in Js:
        newJs.append(J.subs([
        (sy.Derivative(q1(t),t), dq[0, 0]),
        (sy.Derivative(q2(t),t), dq[1, 0]),
        (sy.Derivative(q3(t),t), dq[2, 0]),
        (sy.Derivative(q4(t),t), dq[3, 0]),
        (sy.Derivative(q5(t),t), dq[4, 0]),
        (sy.Derivative(q6(t),t), dq[5, 0]),
        (sy.Derivative(q7(t),t), dq[6, 0]),
    ]))


In [76]:
from sympy.printing import cxxcode
from sympy.utilities.codegen import codegen

def gen_cpp_code(expr, name):
    code_txt = cxxcode(expr, assign_to="out", standard="c++17")
    with open(name+".cpp", "w") as f:
        f.write(code_txt)

def gen_c(expr, name, dir=""):
    [(c_name, c_code), (h_name, c_header)] = codegen(
        name_expr=(name, expr),
        language="C",
        project= name + "project",
        to_files=False
    )
    
    f = open(dir+c_name, 'w')
    f.write(c_code)
    f.close()

    f = open(dir+h_name, 'w')
    f.write(c_header)
    f.close()

names = ["W0", "BR"] + [str(i) for i in range(7)] + ["ee"]

for i, o in enumerate(os):
    gen_c(o, name="phi_"+names[i], dir="src/")
for i, J in enumerate(Jos):
    gen_c(J, name="Jo_"+names[i], dir="src/")
for i, J in enumerate(JRxs):
    gen_c(J, name="JRx_"+names[i], dir="src/")
for i, J in enumerate(JRys):
    gen_c(J, name="JRy_"+names[i], dir="src/")
for i, J in enumerate(JRzs):
    gen_c(J, name="JRz_"+names[i], dir="src/")
for i, J in enumerate(Jos_dot):
    gen_c(J, name="Jo_dot_"+names[i], dir="src/")
for i, J in enumerate(JRxs_dot):
    gen_c(J, name="JRx_dot_"+names[i], dir="src/")
for i, J in enumerate(JRys_dot):
    gen_c(J, name="JRy_dot_"+names[i], dir="src/")
for i, J in enumerate(JRzs_dot):
    gen_c(J, name="JRz_dot_"+names[i], dir="src/")
